# Feeds with No Critical Validation Errors

Ref: [GH issue](https://github.com/cal-itp/data-infra/issues/513)

Since the amount of validation errors can vary wildly from feed-to-feed, I think we should just count a feed if the amount of validation errors that Cal-ITP considers critical are 0 errors for a feed. For static GTFS, the list of validation errors are included in the `views.validation_fact_daily_feed_notices`. You'll want to group the notices by day and filter out the codes that are **not** included in the `views.validation_code_descriptions table`. The feeds that have 0 count of validation errors should be added to the overall count.

### Findings
For this one we should do 3 graphs as follows: 
1. Static GTFS errors 
1. GTFS-RT errors 
1. Static + GTFS-RT 

The GTFS-RT aspect of this is blocked until the pipeline to validate GTFS-RT has been completed.

In [1]:
import altair as alt
import pandas as pd
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

import calitp
from calitp.tables import tbl
from siuba import *

from shared_utils import altair_utils
from shared_utils import geography_utils

alt.themes.enable("fivethirtyeight")
alt.renderers.enable('altair_saver', fmts=['png'])

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


RendererRegistry.enable('altair_saver')

In [2]:
'''
code_descriptions = (tbl.views.validation_code_descriptions()
 >> collect()
)

code_descriptions.to_parquet("./validation_code_descriptions.parquet")
'''
code_descriptions = pd.read_parquet("./validation_code_descriptions.parquet")

In [3]:
# All of these are critical for reporting...52 errors total
code_descriptions.critical_for_reporting.value_counts()

Series([], Name: critical_for_reporting, dtype: int64)

In [4]:
critical_errors = list(code_descriptions.code.unique())

'''
daily_validations = (
    tbl.views.validation_fact_daily_feed_notices()
    >> distinct(_.feed_key, _.calitp_itp_id, _.calitp_url_number,
                _.date, _.code)
    >> collect()
    >> filter(_.code.isin(critical_errors))
)

daily_validations.to_parquet("./daily_validations.parquet")

# Show number of errors within operator
# Earlier df shows across errors, capturing distinct errors
num_errors = (
    tbl.views.validation_fact_daily_feed_notices()
    >> group_by(_.feed_key, _.calitp_itp_id, _.calitp_url_number,
                _.date, _.code)
    >> count(_.code)
    >> collect()
    >> filter(_.code.isin(critical_errors))
)

num_errors.to_parquet("./num_errors.parquet")
'''

daily_validations = pd.read_parquet("./daily_validations.parquet")
num_errors = pd.read_parquet("./num_errors.parquet")

In [5]:
GROUP_COLS = ["feed_key", "date", "calitp_itp_id", "calitp_url_number"]

daily_validations = daily_validations.merge(
    num_errors.rename(
    columns = {"n": "num_errors"}), 
    on = GROUP_COLS + ["code"],
    how = "inner",
    validate = "1:1"
)

daily_validations.head(2)

,feed_key,calitp_itp_id,calitp_url_number,date,code,num_errors
0,-4045042305146455543,296,0,2021-10-23,decreasing_or_equal_shape_distance,2148
1,-7847305992506683453,48,0,2021-08-15,stop_time_timepoint_without_times,34422


In [6]:
daily_errors = (
    geography_utils.aggregate_by_geography(daily_validations, 
                                           group_cols = GROUP_COLS,
                                           sum_cols = ["num_errors"],
                                           count_cols = ["code"])
    .rename(columns = {"code": "critical_errors"})
)

daily_errors.head(2)

,feed_key,date,calitp_itp_id,calitp_url_number,num_errors,critical_errors
0,-4045042305146455543,2021-10-23,296,0,2148,1
1,-7847305992506683453,2021-08-15,48,0,34422,1


In [7]:
# For a given day, grab all the feeds (exclude info about errors)
'''
daily_feed = (
    tbl.views.validation_fact_daily_feed_notices()
    >> distinct(_.feed_key, _.date,
                _.calitp_itp_id, _.calitp_url_number,
               )
    >> collect()
)

daily_feed.to_parquet("./daily_feed.parquet")
'''
daily_feed = pd.read_parquet("./daily_feed.parquet")

In [8]:
df = pd.merge(
    daily_feed,
    daily_errors,
    on = GROUP_COLS,
    how = "left",
    validate = "1:m"
)

df = df.assign(
    critical_errors = df.critical_errors.fillna(0).astype(int),
    num_errors = df.num_errors.fillna(0).astype(int),
    date = pd.to_datetime(df.date),
    total_feeds = df.groupby("date")["feed_key"].transform("count")
)

df.head()

,feed_key,date,calitp_itp_id,calitp_url_number,num_errors,critical_errors,total_feeds
0,18414309913382105,2021-04-15,1,2,40532,3,143
1,2415344669691788437,2021-11-02,200,0,5092,9,180
2,2415344669691788437,2021-09-30,200,0,5724,9,177
3,2415344669691788437,2021-10-24,200,0,5624,9,177
4,7817634405777406718,2021-10-24,284,0,2782,5,177


### Feeds with No Critical Errors

In [9]:
no_errors = (
    geography_utils.aggregate_by_geography(
        df[df.critical_errors==0],
        group_cols = ["date", "total_feeds"],
        nunique_cols = ["feed_key"]
    )
)

no_errors = no_errors.assign(
    pct_no_errors = no_errors.feed_key.divide(no_errors.total_feeds)
)

In [10]:
axis_date_format ="%-m/%-d/%y"

def base_line_chart(df):
    chart = (alt.Chart(df)
             .mark_line()
             .encode(
                 x=alt.X("date", axis=alt.Axis(format=axis_date_format))
             )
            )
    return chart

In [11]:
chart = base_line_chart(no_errors)
             
chart = (chart
         .encode(
             y=alt.Y("feed_key:Q", title="# feeds"),
             color=alt.value(altair_utils.FIVETHIRTYEIGHT_CATEGORY_COLORS[0])
         ).properties(title="# Feeds with No Critical Errors")
        )

chart = altair_utils.preset_chart_config(chart)
chart.save("./no_errors.png")

![no_errors](./no_errors.png)

Sudden drop...why? Is it because feed expired?

In [12]:
chart = base_line_chart(no_errors)
             
chart = (chart
         .encode(
             y=alt.Y("pct_no_errors:Q", title="% feeds", 
                    axis=alt.Axis(format="%")),
             color=alt.value(altair_utils.FIVETHIRTYEIGHT_CATEGORY_COLORS[0])
         ).properties(title="Percent Feeds with No Critical Errors")
        )

chart = altair_utils.preset_chart_config(chart)
chart.save("./no_errors_pct.png")

![no_errors_pct](./no_errors_pct.png)

### Sum of Operator-Unique Errors by Day

* Total number of unique errors over all the operators.
* For each operator, count the number of unique errors. 
* Sum across the entire day.

In [13]:
chart = base_line_chart((df[df.critical_errors > 0]
                         .groupby("date")
                         .agg({"critical_errors": "sum"})
                         .reset_index()
                        ))
chart = (chart
         .encode(
             y=alt.Y("critical_errors:Q", title="Total # Unique Errors"),
             color=alt.value(altair_utils.FIVETHIRTYEIGHT_CATEGORY_COLORS[0])
         ).properties(title="Total Validation Errors Across Operators")
        )

chart = altair_utils.preset_chart_config(chart)
chart.save("./total_validation_errors.png")

![total_unique_errors](./total_validation_errors.png)

### Unique Errors by Day

* Unique validation error codes by day (across all operators)

In [14]:
unique_errors = (
    tbl.views.validation_fact_daily_feed_notices()
    >> distinct(_.date, _.code)
    >> collect()
    >> filter(_.code.isin(critical_errors))
    >> group_by(_.date)
    >> summarize(num_unique_errors = _.code.count())
)

unique_errors = unique_errors.assign(
    date = pd.to_datetime(unique_errors.date)
)

unique_errors.head()

,date,num_unique_errors
0,2021-04-15,14
1,2021-04-16,15
2,2021-04-17,16
3,2021-04-18,16
4,2021-04-19,16


In [15]:
chart = base_line_chart(unique_errors)

chart = (chart
         .encode(
             y=alt.Y("num_unique_errors:Q", title="# Unique Errors"),
             color=alt.value(altair_utils.FIVETHIRTYEIGHT_CATEGORY_COLORS[0])
         ).properties(title="Unique Validation Errors Across Operators")
        )

chart = altair_utils.preset_chart_config(chart)
chart.save("./unique_validation_errors.png")

![unique_validation_errors](./unique_validation_errors.png)

### Feeds by Error Type
% daily feeds for each type of error

In [16]:
errors_by_type = (
    geography_utils.aggregate_by_geography(
        daily_validations,
        group_cols = ["date", "code"],
        nunique_cols = ["feed_key"]
    ).rename(columns = {"feed_key": "num_feeds"})
)

errors_by_type.head()

,date,code,num_feeds
0,2021-10-23,decreasing_or_equal_shape_distance,31
1,2021-08-15,stop_time_timepoint_without_times,21
2,2021-10-23,stop_time_timepoint_without_times,22
3,2021-05-14,stop_time_timepoint_without_times,24
4,2021-07-01,stop_time_timepoint_without_times,24


In [17]:
errors_by_type2 = pd.merge(
    errors_by_type.assign(date = pd.to_datetime(errors_by_type.date)),
    df[["date", "total_feeds"]].drop_duplicates().reset_index(drop=True), 
    on = "date",
    how = "inner",
    validate = "m:1"
)

errors_by_type2 = errors_by_type2.assign(
    pct_error = errors_by_type2.num_feeds.divide(errors_by_type2.total_feeds)
)

errors_by_type2.head(2)

,date,code,num_feeds,total_feeds,pct_error
0,2021-10-23,decreasing_or_equal_shape_distance,31,177,0.175141
1,2021-10-23,stop_time_timepoint_without_times,22,177,0.124294


In [18]:
chart = base_line_chart(errors_by_type2)

chart = (chart
 .encode(
     y=alt.Y("pct_error:Q", title="% feeds", axis=alt.Axis(format="%")),
     color=alt.Color("code:N", title="Validation Error", 
                    scale=alt.Scale(range=altair_utils.FIVETHIRTYEIGHT_CATEGORY_COLORS))
 ).properties(title="% Daily Feeds by Critical Validation Error Types")
)

chart = (altair_utils.preset_chart_config(chart)
        )
chart.save("./pct_feeds_by_error.png")

![feeds_by_error](./pct_feeds_by_error.png)

In [19]:
# Better chart...pick out the most critical ones and plot those
# 52 is too many
top_errors = (errors_by_type2.groupby(["code"])
              .agg({"num_feeds": "sum"})
              .reset_index()
              .sort_values("num_feeds", ascending=False)
              .reset_index(drop=True)
)

TOP_ERRORS = list(top_errors.code.iloc[:10])
TOP_ERRORS

['duplicate_route_name',
 'decreasing_or_equal_shape_distance',
 'too_fast_travel',
 'stop_time_timepoint_without_times',
 'feed_expiration_date',
 'duplicate_fare_rule_zone_id_fields',
 'route_short_name_too_long',
 'stop_too_far_from_trip_shape',
 'decreasing_or_equal_stop_time_distance',
 'same_name_and_description_for_route']

In [20]:
chart = base_line_chart(
    errors_by_type2[errors_by_type2.code.isin(TOP_ERRORS)])

chart = (chart
 .encode(
     y=alt.Y("pct_error:Q", title="% feeds", axis=alt.Axis(format="%")),
     color=alt.Color("code:N", title="Validation Error", 
                    scale=alt.Scale(range=altair_utils.FIVETHIRTYEIGHT_CATEGORY_COLORS))
 ).properties(title="% Daily Feeds by Top 10 Critical Validation Error Types")
)

chart = (altair_utils.preset_chart_config(chart)
        )

chart.save("./pct_feeds_by_error_top10.png")

![top10_errors](./pct_feeds_by_error_top10.png)

In [21]:
# Aggregating by error type across all dates available
top_errors

,code,num_feeds
0,duplicate_route_name,8065
1,decreasing_or_equal_shape_distance,7278
2,too_fast_travel,6129
3,stop_time_timepoint_without_times,5509
4,feed_expiration_date,4374
5,duplicate_fare_rule_zone_id_fields,4229
6,route_short_name_too_long,3551
7,stop_too_far_from_trip_shape,3366
8,decreasing_or_equal_stop_time_distance,1406
9,same_name_and_description_for_route,1136


Metabase Q: which feeds are expired? Click on a validation error and see which operators.